In [15]:
import pandas as pd 
import numpy as np


import pandasql as psql
#machine learning and statistical methods

#selected preprocessing and evaluation methods

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit

In [16]:
formtemplatesclones=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesClones.csv",parse_dates=["date"])
formtemplatescategories=pd.read_csv("/home/batuhan-saylam/Desktop/JotformProject/formTemplatesCategories.csv")
formtemplates=pd.read_excel("/home/batuhan-saylam/Desktop/JotformProject/formTemplates_1_.xlsx")

In [29]:
len(formtemplatescategories.metaKeywords.unique().tolist())

292

In [18]:
formtemplatescategoriesy1=formtemplatescategories.loc[formtemplatescategories["metaKeywords"]== 'Summer Camp Evaluations',:].copy()
formtemplatescategoriesy2=formtemplatescategories.loc[formtemplatescategories["metaKeywords"]== 'Summer Camp Medical Forms',:].copy()
formtemplatescategoriesy3=formtemplatescategories.loc[formtemplatescategories["metaKeywords"]== 'Summer Camp Consent Forms',:].copy()
formtemplatescategoriesy4=formtemplatescategories.loc[formtemplatescategories["metaKeywords"]== 'Summer Camp Job Appl Forms',:].copy()
formtemplatescategoriesy5=formtemplatescategories.loc[formtemplatescategories["metaKeywords"]==  
                                                      'Summer, Camp, Application, Forms, summer camp application forms, summer camp application form, sample summer camp application form',:].copy()

formtemplatescategoriesy=pd.concat([formtemplatescategoriesy1,formtemplatescategoriesy2,formtemplatescategoriesy3,formtemplatescategoriesy4,formtemplatescategoriesy5])

In [19]:
formtemplatescategoriesy

,_id,name,slug,description,pageTitle,metaKeywords
4,1049599044,Summer Camp Surveys,summer-camp-surveys,Gather feedback from campers and their parents...,NaN,Summer Camp Evaluations
5,1069140533,Summer Camp Medical Forms,summer-camp-medical-forms,Collect campers' medical information with secu...,NaN,Summer Camp Medical Forms
6,1069207537,Summer Camp Consent Forms,summer-camp-consent-forms,"Free consent forms, waivers, and permission sl...",NaN,Summer Camp Consent Forms
239,9210371232,Summer Camp Application Forms,summer-camp-application-forms,"Start the summer off right with our free, cust...",NaN,"Summer, Camp, Application, Forms, summer camp ..."


In [20]:

formtemplatesclones=formtemplatesclones.rename({"templateID":"_id"},axis=1)
formandclone=pd.merge(formtemplates,formtemplatesclones,how="inner",on="_id")
formtemplates1=formtemplates.copy()
formtemplates1=formtemplates1.rename({"_id":"id"},axis=1)
formtemplates1=formtemplates1.rename({"_featuredCategory":"_id"},axis=1)
df1_melt = formtemplates1.assign(_categories=formtemplates1._categories.str.split(","))
formtemplates1=df1_melt._categories.apply(pd.Series) \
    .merge(formtemplates1, right_index=True, left_index=True) \
    .drop(["_categories"], axis=1) \
    .melt(id_vars=['id',"_title","_slug","_description","_id","_language"], value_name="_categories") \
    .drop("variable", axis=1) 
formtemplates2=pd.merge(formtemplatescategoriesy,formtemplates1,how="inner",on="_id")
formtemplates2=formtemplates2.rename({"_id":"_featuredCategory"},axis=1)
formtemplates1=formtemplates1.rename({"_id":"_featuredCategory"},axis=1)
formtemplates1=formtemplates1.rename({"_categories":"_id"},axis=1)
formtemplates3=pd.merge(formtemplatescategoriesy,formtemplates1,how="inner",on="_id")
formtemplates3=formtemplates3.rename({"_id":"_categories"},axis=1)
formtemplates1=formtemplates1.rename({"_id":"_categories"},axis=1)
formtemplates1=formtemplates1.rename({"id":"_id"},axis=1)
formtemplatesclones=formtemplatesclones.rename({"_id":"templateID"},axis=1)
formtemplates2=formtemplates2.drop_duplicates()
formtemplates3=formtemplates3.drop_duplicates()
formcat=pd.concat([formtemplates2,formtemplates3]).drop_duplicates()
formcat=formcat.rename({"id":"_id"},axis=1)
formcat=formcat.loc[:,[
    'name',
    'description',
    'metaKeywords',
    '_categories',
    '_featuredCategory',
    '_id']]
formandclone=formandclone.loc[:,[
'_id',
    '_title',
    '_slug',
    '_description',
    '_language',
    'formID',
    'form_type',
    'source',
    'date']]

formclonecategnonnany=pd.merge(formandclone,formcat,on="_id")
formclonecategnonnany=formclonecategnonnany.drop_duplicates(keep='first')
formclonecategnonnany.index=formclonecategnonnany.date
formclonecategnonnany.index=formclonecategnonnany.index.to_period("D")
formclonecategnonnany=formclonecategnonnany.rename({"date":"date2"},axis=1)
formclonecategnonnany.index=formclonecategnonnany.index.to_timestamp()

In [21]:
df=psql.sqldf("select count(_id),date from formclonecategnonnany  group by date")
df=df.loc[:len(df)-2,]
df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])
df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]
df=df.resample('7D',on="ds").sum()
decomposeresult2=seasonal_decompose(df["y"],period=52,model="add",extrapolate_trend="freq",two_sided=True)
season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
season=season.fillna(0)
season=season.sort_index()
season.index=season.index.to_period("D")
season.index=season.index.to_timestamp()
season.index=season.index.strftime('%d-%m-%Y')
season.index=pd.to_datetime(season.index,format='%d-%m-%Y')

In [22]:
season

,obs,trend,seasonal,resid
ds,,,,
2020-01-01,109.0,161.656944,-124.314345,71.657400
2020-01-08,162.0,162.980016,-65.765305,64.785289
2020-01-15,179.0,164.303087,-51.569630,66.266543
2020-01-22,247.0,165.626158,-30.428100,111.801942
2020-01-29,252.0,166.949230,-6.139891,91.190662
...,...,...,...,...
2023-06-14,517.0,411.905239,162.794956,-57.700195
2023-06-21,581.0,414.265872,167.090376,-0.356248
2023-06-28,402.0,416.626504,76.195669,-90.822173


In [23]:
fig=make_subplots(rows=4, cols=1,
                subplot_titles=("summer camp", "Trend", "Seasonal_yearly","resid"))
fig.add_trace(
    go.Scatter(x=season.index,y=season.obs),row=1,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.trend),row=2,col=1
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.resid),row=4,col=1
    
)


fig.update_layout(height=1000, width=950)

In [24]:
df=psql.sqldf("select count(_id),date from formclonecategnonnany  group by date")
df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])
df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]
df["roll"]=df["y"].rolling(7).mean()
df.dropna(inplace=True)
decomposeresult2=seasonal_decompose(df["roll"],period=365,model="add",extrapolate_trend="freq",two_sided=True)
season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
season=season.fillna(0)
season=season.sort_index()
season.index=season.index.to_period("D")
season.index=season.index.to_timestamp()
season.index=season.index.strftime('%d-%m-%Y')
season.index=pd.to_datetime(season.index,format='%d-%m-%Y')

In [25]:
df

,y,ds,roll
07-01-2020,28,2020-01-07,15.571429
08-01-2020,29,2020-01-08,18.857143
09-01-2020,28,2020-01-09,21.285714
10-01-2020,20,2020-01-10,21.428571
11-01-2020,16,2020-01-11,21.857143
...,...,...,...
15-07-2023,49,2023-07-15,58.142857
16-07-2023,41,2023-07-16,57.571429
17-07-2023,72,2023-07-17,59.142857
18-07-2023,66,2023-07-18,60.000000


In [26]:
fig=make_subplots(rows=4, cols=1,
                subplot_titles=("summer camp", "Trend", "Seasonal_yearly","resid"))
fig.add_trace(
    go.Scatter(x=season.index,y=season.obs),row=1,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.trend),row=2,col=1
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.resid),row=4,col=1
    
)


fig.update_layout(height=1000, width=950)

In [27]:
df=psql.sqldf("select count(_id),date from formclonecategnonnany  group by date")
df.index=df.date.tolist()
df.index=pd.to_datetime(df.index)
df.index=df.index.to_period("D")
df.index=df.index.to_timestamp()
df.index=df.index.strftime('%d-%m-%Y')
df["date"]=pd.to_datetime(df["date"])
df=df.rename({"count(_id)":"y"},axis=1)
df=df.rename({"date":"ds"},axis=1)
df=df.loc[:,["y","ds"]]
df["roll"]=df["y"].rolling(7).sum()
df.dropna(inplace=True)
decomposeresult2=seasonal_decompose(df["roll"],period=365,model="add",extrapolate_trend="freq",two_sided=True)
season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal,"resid":decomposeresult2.resid})
season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
season=season.fillna(0)
season=season.sort_index()
season.index=season.index.to_period("D")
season.index=season.index.to_timestamp()
season.index=season.index.strftime('%d-%m-%Y')
season.index=pd.to_datetime(season.index,format='%d-%m-%Y')

In [28]:
fig=make_subplots(rows=4, cols=1,
                subplot_titles=("summer camp", "Trend", "Seasonal_yearly","resid"))
fig.add_trace(
    go.Scatter(x=season.index,y=season.obs),row=1,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.trend),row=2,col=1
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.seasonal),row=3,col=1
    
)
fig.add_trace(
    go.Scatter(x=season.index,y=season.resid),row=4,col=1
    
)


fig.update_layout(height=1000, width=950)